In [129]:
import pandas as pd
import numpy as np
cardata = pd.read_csv('ebay_car_scrapped_dataset_cleaned_withmoredata.csv')
cardata['Year'] = cardata['Year'].apply(str)

In [87]:
cardata.shape

(2946, 20)

In [88]:
cardata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2946 entries, 0 to 2945
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Year                        2946 non-null   object
 1   Mileage                     2946 non-null   int64 
 2   Fuel Type                   2946 non-null   object
 3   Vehicle Title               2946 non-null   object
 4   Condition                   2946 non-null   object
 5   Report Car Name             2946 non-null   object
 6   Vehicle Age                 2946 non-null   int64 
 7   Calculated Owners           2946 non-null   int64 
 8   State Title Brand Reported  2946 non-null   object
 9   Accidents Reported          2946 non-null   int64 
 10  Odometer                    2946 non-null   object
 11  Open Recall                 2946 non-null   object
 12  Number of Cylinders         2946 non-null   object
 13  Engine                      2946 non-null   obje

In [130]:
traindata = pd.get_dummies(cardata.drop('Car Price',axis=1))

In [131]:
testdata = cardata['Car Price']

In [91]:
testdata.value_counts()

0 - 20k      911
20k - 40k    826
80k+         516
40k - 60k    399
60k - 80k    294
Name: Car Price, dtype: int64

In [92]:
from sklearn.metrics import classification_report

In [93]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(traindata,testdata,test_size=0.2,random_state=101)

In [95]:
from sklearn.ensemble import RandomForestClassifier
rd = RandomForestClassifier()
rd.fit(xtrain,ytrain)
pred = rd.predict(xtest)
print(classification_report(ytest,pred))

              precision    recall  f1-score   support

     0 - 20k       0.78      0.84      0.81       178
   20k - 40k       0.65      0.68      0.66       170
   40k - 60k       0.57      0.46      0.51        80
   60k - 80k       0.70      0.54      0.61        61
        80k+       0.81      0.86      0.84       101

    accuracy                           0.72       590
   macro avg       0.70      0.68      0.69       590
weighted avg       0.71      0.72      0.71       590



In [96]:
traindata = pd.get_dummies(cardata.drop(['VIN','Make','Car Price','Report Car Name'],axis=1))

In [97]:
testdata = cardata['Car Price']

In [99]:
testdata.value_counts()

0 - 20k      911
20k - 40k    826
80k+         516
40k - 60k    399
60k - 80k    294
Name: Car Price, dtype: int64

In [98]:
#base model
xtrain,xtest,ytrain,ytest = train_test_split(traindata,testdata,test_size=0.2,random_state=101)

rd = RandomForestClassifier()
rd.fit(xtrain,ytrain)
pred = rd.predict(xtest)
print(classification_report(ytest,pred))

              precision    recall  f1-score   support

     0 - 20k       0.78      0.82      0.80       178
   20k - 40k       0.63      0.67      0.65       170
   40k - 60k       0.54      0.45      0.49        80
   60k - 80k       0.55      0.48      0.51        61
        80k+       0.77      0.78      0.77       101

    accuracy                           0.68       590
   macro avg       0.65      0.64      0.64       590
weighted avg       0.68      0.68      0.68       590



In [132]:
#after changing categorical variables into 3 sets
cardata = pd.read_csv('ebay_car_scrapped_dataset_cleaned_withmoredata.csv')
cardata['Year'] = cardata['Year'].apply(str)
cardata['Car Price'].replace({'80k+':3, '40k - 60k':2, '0 - 20k':1, '20k - 40k':2, '60k - 80k':3},inplace=True)
traindata = pd.get_dummies(cardata.drop(['VIN','Make','Car Price','Report Car Name'],axis=1))
testdata = cardata['Car Price']

In [133]:
#base model
xtrain,xtest,ytrain,ytest = train_test_split(traindata,testdata,test_size=0.2,random_state=101)

rd = RandomForestClassifier()
rd.fit(xtrain,ytrain)
pred = rd.predict(xtest)
print(classification_report(ytest,pred))

              precision    recall  f1-score   support

           1       0.81      0.80      0.80       178
           2       0.74      0.77      0.75       250
           3       0.84      0.80      0.82       162

    accuracy                           0.79       590
   macro avg       0.80      0.79      0.79       590
weighted avg       0.79      0.79      0.79       590



In [134]:
#parameter tuning

from sklearn.model_selection import RandomizedSearchCV
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 100, num = 10)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
rf_random = RandomizedSearchCV(estimator = RandomForestClassifier(), param_distributions = random_grid,
                               n_iter = 100, cv = 5, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(xtrain,ytrain)
print(classification_report(ytest,rf_random.predict(xtest)))

Fitting 5 folds for each of 100 candidates, totalling 500 fits
              precision    recall  f1-score   support

           1       0.83      0.79      0.81       178
           2       0.74      0.80      0.77       250
           3       0.85      0.80      0.82       162

    accuracy                           0.79       590
   macro avg       0.81      0.79      0.80       590
weighted avg       0.80      0.79      0.79       590



In [136]:
rf_random.best_params_

{'n_estimators': 1800,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 20,
 'bootstrap': False}

In [137]:
r = RandomForestClassifier(n_estimators = 1800,
 min_samples_split = 5,
 min_samples_leaf = 1,
 max_features = 'sqrt',
 max_depth = 20,
 bootstrap = False)

In [135]:
rd.feature_importances_

array([2.19305672e-01, 9.02548403e-02, 4.26727037e-02, 3.18934045e-02,
       5.63220339e-03, 4.32073314e-03, 5.38343903e-03, 5.58338063e-03,
       7.08055269e-03, 6.62698153e-03, 8.22490747e-03, 8.14638680e-03,
       7.91232512e-03, 1.06957794e-02, 1.01899263e-02, 1.14773965e-02,
       1.83078473e-02, 1.03075063e-04, 8.79242063e-03, 4.15687934e-03,
       5.06153013e-03, 1.13017722e-02, 2.05576197e-03, 1.46894278e-02,
       1.46623537e-03, 3.07025020e-04, 6.31130216e-03, 8.67171648e-03,
       3.30379745e-03, 3.64881834e-03, 4.66766029e-03, 1.34072897e-02,
       1.54044351e-02, 3.52493127e-03, 3.87094434e-03, 1.22135062e-02,
       1.29964739e-02, 1.75291249e-02, 5.49513348e-04, 2.91505440e-02,
       5.27089380e-03, 0.00000000e+00, 1.32416784e-02, 1.12921741e-03,
       5.36461967e-04, 2.83467799e-04, 4.69339278e-03, 1.02947206e-02,
       1.31214051e-02, 1.25118059e-02, 7.14951811e-03, 9.42398118e-03,
       1.78142379e-02, 7.18353006e-05, 4.04749409e-04, 3.57688717e-03,
      

[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=2000; total time=   4.3s
[CV] END bootstrap=True, max_depth=None, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=1400; total time=   3.8s
[CV] END bootstrap=True, max_depth=20, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=1200; total time=   2.8s
[CV] END bootstrap=False, max_depth=70, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=800; total time=   2.8s
[CV] END bootstrap=True, max_depth=30, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=1800; total time=   4.5s
[CV] END bootstrap=False, max_depth=80, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=1800; total time=   5.1s
[CV] END bootstrap=False, max_depth=90, max_features=auto, min_samples_leaf=4, min_samples_split=2, n_estimators=2000; total time=   5.6s
[CV] END bootstrap=True, max_depth

[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=2000; total time=   4.3s
[CV] END bootstrap=True, max_depth=None, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=1400; total time=   3.8s
[CV] END bootstrap=True, max_depth=50, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=1600; total time=   4.0s
[CV] END bootstrap=False, max_depth=70, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=800; total time=   2.7s
[CV] END bootstrap=False, max_depth=50, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=600; total time=   1.7s
[CV] END bootstrap=False, max_depth=50, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=600; total time=   1.7s
[CV] END bootstrap=False, max_depth=80, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=1800; total time=   5.1s
[CV] END bootstrap=False, max_depth=

[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=2000; total time=   4.3s
[CV] END bootstrap=True, max_depth=None, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=1400; total time=   3.9s
[CV] END bootstrap=True, max_depth=50, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=1600; total time=   4.1s
[CV] END bootstrap=True, max_depth=30, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=1800; total time=   4.5s
[CV] END bootstrap=False, max_depth=90, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=1000; total time=   2.6s
[CV] END bootstrap=False, max_depth=80, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=1800; total time=   5.2s
[CV] END bootstrap=False, max_depth=90, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=2000; total time=   5.9s
[CV] END bootstrap=False, max_dep

[CV] END bootstrap=True, max_depth=60, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=1200; total time=   3.0s
[CV] END bootstrap=True, max_depth=60, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=1200; total time=   3.2s
[CV] END bootstrap=True, max_depth=50, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.6s
[CV] END bootstrap=True, max_depth=20, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=1200; total time=   2.8s
[CV] END bootstrap=True, max_depth=50, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=1600; total time=   4.0s
[CV] END bootstrap=False, max_depth=50, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=600; total time=   1.8s
[CV] END bootstrap=False, max_depth=50, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=600; total time=   1.7s
[CV] END bootstrap=False, max_depth=90, m

[CV] END bootstrap=True, max_depth=60, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=1200; total time=   3.0s
[CV] END bootstrap=True, max_depth=60, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=1200; total time=   3.2s
[CV] END bootstrap=True, max_depth=50, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.6s
[CV] END bootstrap=True, max_depth=20, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=1200; total time=   2.9s
[CV] END bootstrap=False, max_depth=70, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=800; total time=   2.8s
[CV] END bootstrap=True, max_depth=30, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=1800; total time=   4.5s
[CV] END bootstrap=False, max_depth=90, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=1000; total time=   2.5s
[CV] END bootstrap=False, max_depth=90, 

In [142]:
from sklearn.feature_selection import SelectFromModel
from numpy import sort

thresholds = sort(rd.feature_importances_)
max_accuracy = 0
thre = 0
features = 0
for thresh in thresholds:
    # select features using threshold
    selection = SelectFromModel(rd, threshold=thresh, prefit=True)
    select_X_train = selection.transform(xtrain)
    # train model
    selection_model = RandomForestClassifier(n_estimators = 1800,
 min_samples_split = 5,
 min_samples_leaf = 1,
 max_features = 'sqrt',
 max_depth = 20,
 bootstrap = False)
    selection_model.fit(select_X_train, ytrain)
    # eval model
    select_X_test = selection.transform(xtest)
    y_pred = selection_model.predict(select_X_test)
    if(classification_report(ytest,y_pred,output_dict=True)['accuracy']>max_accuracy):
        max_accuracy = classification_report(ytest,y_pred,output_dict=True)['accuracy']
        thre = thresh
        features = select_X_train.shape[1]
print(max_accuracy)
print(thre)
print(features)

/Users/harita_addanki/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/harita_addanki/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/harita_addanki/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/harita_addanki/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/harita_addanki/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/harita_addanki/opt/anaconda3/lib/python3.9/site-

/Users/harita_addanki/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/harita_addanki/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/harita_addanki/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/harita_addanki/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/harita_addanki/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/harita_addanki/opt/anaconda3/lib/python3.9/site-

/Users/harita_addanki/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/harita_addanki/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/harita_addanki/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/harita_addanki/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/harita_addanki/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/harita_addanki/opt/anaconda3/lib/python3.9/site-

/Users/harita_addanki/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/harita_addanki/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/harita_addanki/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/harita_addanki/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/harita_addanki/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/harita_addanki/opt/anaconda3/lib/python3.9/site-

0.8050847457627118
0.007080552693229671
42


/Users/harita_addanki/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


In [106]:
from sklearn.feature_selection import RFE
rfe_selector = RFE(estimator=rf_random, n_features_to_select=850, step=5, verbose=5)
rfe_selector.fit(xtrain,ytrain)
rfe_support = rfe_selector.get_support()
rfe_feature = xtrain.loc[:,rfe_support].columns.tolist()
rf = RandomForestClassifier()
rf.fit(xtrain[rfe_feature],ytrain)
predictions = rf.predict(xtest[rfe_feature])
print(classification_report(ytest,predictions))

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=2000; total time=   4.3s
[CV] END bootstrap=True, max_depth=50, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.6s
[CV] END bootstrap=True, max_depth=50, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.6s
[CV] END bootstrap=True, max_depth=50, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.6s
[CV] END bootstrap=True, max_depth=50, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.6s
[CV] END bootstrap=True, max_depth=20, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=1200; total time=   2.8s
[CV] END bootstrap=True, max_depth=50, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=1600;

/Users/harita_addanki/opt/anaconda3/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


              precision    recall  f1-score   support

           1       0.80      0.79      0.79       178
           2       0.74      0.76      0.75       250
           3       0.83      0.80      0.82       162

    accuracy                           0.78       590
   macro avg       0.79      0.78      0.79       590
weighted avg       0.78      0.78      0.78       590

[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=2000; total time=   4.3s
[CV] END bootstrap=True, max_depth=None, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=1400; total time=   3.4s
[CV] END bootstrap=True, max_depth=50, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=1600; total time=   3.9s
[CV] END bootstrap=False, max_depth=70, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=800; total time=   2.7s
[CV] END bootstrap=False, max_depth=50, max_features=sqrt, min_sampl

[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=2000; total time=   4.3s
[CV] END bootstrap=True, max_depth=None, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=1400; total time=   3.5s
[CV] END bootstrap=True, max_depth=50, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=1600; total time=   3.9s
[CV] END bootstrap=True, max_depth=30, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=1800; total time=   4.6s
[CV] END bootstrap=False, max_depth=90, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=1000; total time=   2.5s
[CV] END bootstrap=False, max_depth=80, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=1800; total time=   5.0s
[CV] END bootstrap=False, max_depth=90, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=2000; total time=   5.2s
[CV] END bootstrap=False, max_dep

[CV] END bootstrap=True, max_depth=60, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=1200; total time=   3.0s
[CV] END bootstrap=True, max_depth=None, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=1400; total time=   3.4s
[CV] END bootstrap=True, max_depth=20, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=1200; total time=   2.8s
[CV] END bootstrap=False, max_depth=70, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=800; total time=   2.7s
[CV] END bootstrap=True, max_depth=30, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=1800; total time=   4.5s
[CV] END bootstrap=False, max_depth=90, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=1000; total time=   2.5s
[CV] END bootstrap=False, max_depth=90, max_features=auto, min_samples_leaf=4, min_samples_split=2, n_estimators=2000; total time=   5.0s
[CV] END bootstrap=False, max_depth=

[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=2000; total time=   4.3s
[CV] END bootstrap=True, max_depth=None, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=1400; total time=   3.4s
[CV] END bootstrap=True, max_depth=20, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=1200; total time=   2.8s
[CV] END bootstrap=False, max_depth=70, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=800; total time=   2.7s
[CV] END bootstrap=True, max_depth=30, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=1800; total time=   4.5s
[CV] END bootstrap=False, max_depth=80, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=1800; total time=   5.0s
[CV] END bootstrap=False, max_depth=90, max_features=auto, min_samples_leaf=4, min_samples_split=2, n_estimators=2000; total time=   5.0s
[CV] END bootstrap=True, max_depth

In [104]:
from sklearn.feature_selection import SelectFromModel
from numpy import sort

thresholds = sort(rd.feature_importances_)
max_accuracy = 0
thre = 0
features = 0
for thresh in thresholds:
    # select features using threshold
    selection = SelectFromModel(rd, threshold=thresh, prefit=True)
    select_X_train = selection.transform(xtrain)
    # train model
    selection_model = RandomForestClassifier()
    selection_model.fit(select_X_train, ytrain)
    # eval model
    select_X_test = selection.transform(xtest)
    y_pred = selection_model.predict(select_X_test)
    if(classification_report(ytest,y_pred,output_dict=True)['accuracy']>max_accuracy):
        max_accuracy = classification_report(ytest,y_pred,output_dict=True)['accuracy']
        thre = thresh
        features = select_X_train.shape[1]
print(max_accuracy)
print(thre)
print(features)

/Users/harita_addanki/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/harita_addanki/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/harita_addanki/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/harita_addanki/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/harita_addanki/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/harita_addanki/opt/anaconda3/lib/python3.9/site-

/Users/harita_addanki/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/harita_addanki/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/harita_addanki/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/harita_addanki/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/harita_addanki/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/harita_addanki/opt/anaconda3/lib/python3.9/site-

/Users/harita_addanki/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/harita_addanki/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/harita_addanki/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/harita_addanki/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/harita_addanki/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/harita_addanki/opt/anaconda3/lib/python3.9/site-

/Users/harita_addanki/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/harita_addanki/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/harita_addanki/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/harita_addanki/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/harita_addanki/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/harita_addanki/opt/anaconda3/lib/python3.9/site-

0.8169491525423729
0.008975418913991817
36


/Users/harita_addanki/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


In [103]:
from sklearn.feature_selection import RFE
rfe_selector = RFE(estimator=RandomForestClassifier(), n_features_to_select=850, step=5, verbose=5)
rfe_selector.fit(xtrain,ytrain)
rfe_support = rfe_selector.get_support()
rfe_feature = xtrain.loc[:,rfe_support].columns.tolist()
rf = RandomForestClassifier()
rf.fit(xtrain[rfe_feature],ytrain)
predictions = rf.predict(xtest[rfe_feature])
print(classification_report(ytest,predictions))

              precision    recall  f1-score   support

           1       0.80      0.79      0.80       178
           2       0.74      0.77      0.75       250
           3       0.84      0.80      0.82       162

    accuracy                           0.78       590
   macro avg       0.79      0.79      0.79       590
weighted avg       0.79      0.78      0.79       590



In [117]:
import pandas as pd
import numpy as np
data = pd.read_csv('ebay_car_scrapped_dataset_cleaned_initial.csv')
data['Year'] = data['Year'].apply(str)

In [111]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2946 entries, 0 to 2945
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Year                        2946 non-null   object
 1   Mileage                     2946 non-null   int64 
 2   Fuel Type                   2946 non-null   object
 3   Vehicle Title               2946 non-null   object
 4   Condition                   2946 non-null   object
 5   Report Car Name             2946 non-null   object
 6   Vehicle Age                 2946 non-null   int64 
 7   Calculated Owners           2946 non-null   int64 
 8   State Title Brand Reported  2946 non-null   object
 9   Accidents Reported          2946 non-null   int64 
 10  Odometer                    2946 non-null   object
 11  Open Recall                 2946 non-null   object
 12  Number of Cylinders         2946 non-null   object
 13  Engine                      2946 non-null   obje

In [118]:
data.drop('Car Price',axis=1,inplace=True)

In [121]:
traindata = pd.get_dummies(data.drop('Car Price Original',axis=1))
testdata = data['Car Price Original']

In [122]:
#base model
xtrain,xtest,ytrain,ytest = train_test_split(traindata,testdata,test_size=0.2,random_state=101)

rd = RandomForestClassifier()
rd.fit(xtrain,ytrain)
pred = rd.predict(xtest)
print(classification_report(ytest,pred))

              precision    recall  f1-score   support

        1000       0.56      0.83      0.67         6
        1025       0.00      0.00      0.00         0
        1999       0.00      0.00      0.00         0
        2000       0.00      0.00      0.00         1
        2100       0.00      0.00      0.00         0
        2370       0.00      0.00      0.00         1
        3000       0.00      0.00      0.00         0
        3050       0.00      0.00      0.00         1
        3300       0.00      0.00      0.00         0
        3500       0.00      0.00      0.00         1
        3950       0.00      0.00      0.00         0
        4950       0.00      0.00      0.00         1
        4999       0.00      0.00      0.00         1
        5000       0.00      0.00      0.00         3
        5100       0.00      0.00      0.00         0
        5200       0.00      0.00      0.00         0
        5300       0.00      0.00      0.00         1
        5500       0.00    

/Users/harita_addanki/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/harita_addanki/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/harita_addanki/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
